# financial_ratios.jsonl

### Importing Libraries and Reading Data

In [ ]:
import pandas as pd
import json

In [ ]:
filepath = "financial_ratios.jsonl"
df_json = pd.read_json(filepath, lines=True, dtype={'cust_num': str})
df_json

### Cleaning

In [ ]:
print(df_json.info())
df_json.head()

In [ ]:
# Step 1: Identify numeric columns stored as object
num_cols = [
    "monthly_income",
    "existing_monthly_debt",
    "monthly_payment",
    "revolving_balance",
    "credit_usage_amount",
    "available_credit",
    "total_monthly_debt_payment",
    "total_debt_amount",
    "monthly_free_cash_flow"
]

# Step 2: Clean symbols (remove $, commas, spaces)
for col in num_cols:
    df_json[col] = (
        df_json[col]
        .astype(str)
        .str.replace("$", "", regex=False)
        .str.replace(",", "", regex=False)
        .str.replace(" ", "", regex=False)
    )

# Step 3: Convert to numeric
for col in num_cols:
    df_json[col] = pd.to_numeric(df_json[col], errors="coerce")

In [ ]:
mask = df_json['revolving_balance'].notnull()

# Compare only rows where revolving_balance is not null
(df_json.loc[mask, 'revolving_balance'] == df_json.loc[mask, 'credit_usage_amount']).all()

In [ ]:
df_json['revolving_balance'] = df_json['revolving_balance'].fillna(
    df_json['credit_usage_amount']
)

In [ ]:
df_json.info()

# geographic_data.xml

### Importing Libraries and Reading Data

In [ ]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET

In [ ]:
path = "geographic_data.xml"
tree = ET.parse(path)
root = tree.getroot()

xml_rows = []
for child in root:
    row = {}
    for elem in child:
        row[elem.tag] = elem.text
    xml_rows.append(row)

df_xml = pd.DataFrame(xml_rows)
df_xml.info()

### Cleaning

In [ ]:
#print(df_xml.info())
df_xml = df_xml.apply(lambda col: col.str.strip())
num_cols = ["regional_unemployment_rate", "regional_median_income", "regional_median_rent", "housing_price_index", "cost_of_living_index"]
for col in num_cols:
    df_xml[col] = (
        df_xml[col]
        .astype(str).str.strip()          
        .str.replace(",", "").str.replace("%", "") 
    )
    df_xml[col] = pd.to_numeric(df_xml[col], errors="coerce")
print(df_xml.info())
df_xml.head()

In [ ]:
df_xml['state'].unique() #all good

In [ ]:
df_xml.info()

# loan_details.xlsx

### Importing Libraries and Reading Data

In [ ]:
excel_path = "loan_details.xlsx"
df_excel = pd.read_excel(excel_path, dtype={'customer_id': 'str', 'loan_officer_id': 'str'} )
print(df_excel.info())
df_excel.head(2)

### Cleaning

In [ ]:
print(df_excel['loan_type'].unique())
df_excel['loan_type'] = ( df_excel['loan_type'].str.strip().str.lower().str.replace('personal loan', 'personal')
                                                           .str.replace('creditcard', 'credit card')
                                                           .str.replace('cc', 'credit card'))
df_excel['loan_type'].unique()

In [ ]:
df_excel['loan_amount'] = (
    df_excel['loan_amount']
    .str.strip()
    .str.replace('$', '', regex=False)
    .str.replace(',', '', regex=False)
    .str.replace(r'[$€£¥₹₽₩₺₴₦₱₫₲₵₡₮₢₨]', '', regex=True)
)
df_excel['loan_amount'] = pd.to_numeric(df_excel['loan_amount'], errors='coerce')
print(df_excel['loan_amount'].info())
df_excel.head(2)

In [ ]:
a = df_excel['loan_term'].to_list()
a  # all good
b = df_excel['interest_rate'] < 0
b.unique() # all good
df_excel['loan_purpose'].unique() # all good
df_excel['loan_to_value_ratio'].info() # all good
df_excel['origination_channel'].unique() ## all good
df_excel['marketing_campaign'].unique() # all good

In [ ]:
print(df_excel.info())
df_excel.head(2)

# credit_history.parquet

### Importing Libraries and Reading Data

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
parquet_path = "credit_history.parquet"
df_parquet = pd.read_parquet(parquet_path)
df_parquet['customer_number'] = df_parquet['customer_number'].astype(str)
df_parquet.head()

In [ ]:
df_parquet.info()
# 832(0.92%) null values in num_delinquencies_2yrs column

### Cleaning

In [ ]:
dup_count = df_parquet.duplicated(subset=['customer_number']).sum()
print(f'Total duplicate customer_number rows: {dup_count}')

In [ ]:
df_parquet['num_delinquencies_2yrs'].describe()

In [ ]:
df_parquet.isnull().sum()

In [ ]:
value_counts = df_parquet['num_delinquencies_2yrs'].value_counts().sort_index()
print(value_counts)
# 0 - 97,17%
# 1 - 1,88%
# 2 - 0,01%
# null - 0,92%

In [ ]:
# Fill missing values with 0
df_parquet['num_delinquencies_2yrs'] = df_parquet['num_delinquencies_2yrs'].fillna(0)

In [ ]:
df_parquet.info()

# Metadata.csv

### Importing Libraries and Reading Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [ ]:
csv_path = "pplication_metadata.csv"
df_metadata = pd.read_csv(csv_path, dtype= {'customer_ref': 'str','application_id': 'str'})
df_metadata.head()

In [ ]:
df_metadata.info()

### Cleaning

In [ ]:
df_metadata = df_metadata.rename(columns={'customer_ref':'id'})  

In [ ]:
df_metadata["random_noise_1"] = df_metadata["random_noise_1"].round(3)
col = ['num_login_sessions', "num_customer_service_calls", "has_mobile_app", "paperless_billing", 'application_hour']
a = df_metadata[df_metadata[col] < 0]
a # all good

In [ ]:
df_metadata['default'].isna().sum()

In [ ]:
df_metadata['preferred_contact'].unique()

In [ ]:
df_metadata['preferred_contact'] = df_metadata['preferred_contact'].str.replace("Mail", "Email")
df_metadata['preferred_contact'].unique()

In [ ]:
df_metadata.info()

# Demographics.csv

### Importing Libraries and Reading Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [ ]:
demo_path = "demographics.csv"
df_csv = pd.read_csv(demo_path, dtype={'cust_id': 'str'})
df_csv.head()

In [ ]:
df_csv.info()

### Cleaning

In [ ]:
df_csv.isnull().sum()

In [ ]:
df_csv["employment_type"] = df_csv["employment_type"].str.lower().replace({
    "full_time": "full-time",
    "full time": "full-time",
    "fulltime": "full-time",
    "ft": "full-time",
    
    "part time": "part-time",
    "part_time": "part-time",
    "part-time": "part-time",
    "pt": "part-time",
    
    "self_employed": "self-employed",
    "self emp": "self-employed",
    "self employed": "self-employed",
    "contractor": "contract"
})

In [ ]:
df_csv['employment_length'].describe()

In [ ]:
df_csv['employment_length'] = df_csv['employment_length'].fillna(0)

In [ ]:
df_csv["annual_income"] = (df_csv["annual_income"].str.replace("$", "", regex=False) .str.replace(",", "", regex=False).astype(float))

In [ ]:
df_csv.info()

# Final Dataset

In [ ]:
# Start with metadata as the base
df_final = df_metadata.copy()

# 1. Merge financial_ratios (df_json)
df_final = df_final.merge(
    df_json,
    left_on="id",
    right_on="cust_num",
    how="left"
).drop(columns=["cust_num"])

# 2. Merge geographic_data (df_xml)
df_final = df_final.merge(
    df_xml,
    left_on="id",
    right_on="id",
    how="left"
)

# 3. Merge loan_details (df_excel)
df_final = df_final.merge(
    df_excel,
    left_on="id",
    right_on="customer_id",
    how="left"
).drop(columns=["customer_id"])

# 4. Merge credit_history (df_parquet)
df_final = df_final.merge(
    df_parquet,
    left_on="id",
    right_on="customer_number",
    how="left"
).drop(columns=["customer_number"])

# 5. Merge demographics (df_csv)
df_final = df_final.merge(
    df_csv,
    left_on="id",
    right_on="cust_id",
    how="left"
).drop(columns=["cust_id"])


In [ ]:
cols_to_drop = [
    "existing_monthly_debt",
    "monthly_free_cash_flow",
    "state",
    "regional_unemployment_rate",
    "regional_median_income",
    "regional_median_rent",
    "cost_of_living_index",
    "previous_zip_code",
    "origination_channel",
    "loan_officer_id",
    "marketing_campaign",
    "num_dependents"
]

df_final = df_final.drop(columns=cols_to_drop, errors="ignore")


In [ ]:
df_final.info()

In [ ]:
df_final.to_csv(r'Dataset\final_dataset.csv', index=False)